In [ ]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "coco"
FINAL_DATA_NAME = 'approx1e5-coco-indoor-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "coco-cropped-Indoor"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','coco')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = 0

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [3]:
file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
file_names[:5]

['.DS_Store', 'batch 1', 'batch 3', 'batch 2']

In [4]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [5]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [6]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


batch1-agriVision-RGB-cleaned
batch2-agriVision-RGB-cleaned
toy-agriVision-RGB-cleaned
batch0-agriVision-RGB-cleaned
full-agriVision-RGB-cleaned



# Fourier

In [7]:
#Values obtained from plots in agriVisionFourierEDA.ipynb
STARTING_VALUE = 0.008052940675034493
ENDING_VALUE =0.6162627813472334
MULT_FACTOR = 1.15
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(STARTING_VALUE,ENDING_VALUE, MULT_FACTOR)
print(splits)
BATCH_NUM = None

[0.008052940675034493, 0.009260881776289667, 0.010650014042733115, 0.012247516149143082, 0.014084643571514543, 0.016197340107241723, 0.01862694112332798, 0.021420982291827175, 0.02463412963560125, 0.028329249080941435, 0.03257863644308265, 0.037465431909545044, 0.0430852466959768, 0.04954803370037331, 0.056980238755429305, 0.0655272745687437, 0.07535636575405526, 0.08665982061716354, 0.09965879370973807, 0.11460761276619877, 0.13179875468112856, 0.15156856788329784, 0.1743038530657925, 0.20044943102566135, 0.23051684567951053, 0.2650943725314371, 0.30485852841115263, 0.3505873076728255, 0.4031754038237493, 0.4636517143973116, 0.5331994715569083, 0.6131793922904445, 0.7051563011340111]


In [8]:
TRANSFORM = "fourier"
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()


100%|██████████| 4500/4500 [02:35<00:00, 29.00it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171
 0.53319947 0.61317939 0.7051563 ]


100%|██████████| 33/33 [01:12<00:00,  2.20s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,red,"(0.0, 0.0078125)",10,"[12.37854, 388.76587, -71.011154, -525.40607, ..."
1,2,red,"(0.008052940675034493, 0.008734640537108554)",3,"[22.619307, -0.6036585, 7.497328, 3.4352915, -..."
2,3,red,"(0.009765625, 0.010517900013934578)",3,"[11.679032, -2.0639336, -2.0146694, 0.04964294..."
3,4,red,"(0.011048543456039806, 0.01188039556698871)",4,"[-18.864906, 2.5077233, 4.3224063, -0.9378565,..."
4,5,red,"(0.012352647110032733, 0.014084184669781208)",6,"[-4.914112, 0.8417485, -1.0893494, -1.8732375,..."


In [12]:
del channel_fr, group_data_map, group_data_map_size

In [10]:
TRANSFORM = "fourier"
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [02:33<00:00, 29.29it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171
 0.53319947 0.61317939 0.7051563 ]


100%|██████████| 33/33 [01:08<00:00,  2.09s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,blue,"(0.0, 0.0078125)",10,"[394.00897, -666.8051, -88.63963, -346.68665, ..."
1,2,blue,"(0.008052940675034493, 0.008734640537108554)",3,"[26.316845, -5.964118, 4.277445, -0.005756075,..."
2,3,blue,"(0.009765625, 0.010517900013934578)",3,"[3.8971589, -3.847043, -2.582688, -1.3424733, ..."
3,4,blue,"(0.011048543456039806, 0.01188039556698871)",4,"[-15.663755, 0.2849864, 1.8805587, 0.503372, -..."
4,5,blue,"(0.012352647110032733, 0.014084184669781208)",6,"[-6.3139224, 2.336338, -2.81662, -0.44819397, ..."


In [11]:
del channel_fr, group_data_map, group_data_map_size

In [12]:
TRANSFORM = "fourier"
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [03:02<00:00, 24.70it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171
 0.53319947 0.61317939 0.7051563 ]


100%|██████████| 33/33 [01:25<00:00,  2.60s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,green,"(0.0, 0.0078125)",10,"[166.38116, -696.27155, -199.10657, -450.9094,..."
1,2,green,"(0.008052940675034493, 0.008734640537108554)",3,"[38.9928, -3.1236742, 4.1353054, 0.6367847, -2..."
2,3,green,"(0.009765625, 0.010517900013934578)",3,"[16.542994, -3.9311855, -2.0104222, -1.2822564..."
3,4,green,"(0.011048543456039806, 0.01188039556698871)",4,"[-23.139984, 0.30325156, 2.0036407, 0.14319786..."
4,5,green,"(0.012352647110032733, 0.014084184669781208)",6,"[-3.078197, 2.3377776, -1.9335574, -0.591884, ..."


In [13]:
del channel_fr, group_data_map, group_data_map_size

In [14]:
TRANSFORM = "fourier"
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [04:10<00:00, 17.98it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171
 0.53319947 0.61317939 0.7051563 ]


100%|██████████| 33/33 [01:22<00:00,  2.51s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,gray,"(0.0, 0.0078125)",10,"[146.28271, -368.52505, -148.2057, -461.24997,..."
1,2,gray,"(0.008052940675034493, 0.008734640537108554)",3,"[32.649807, -2.6939397, 5.156004, 1.399945, -2..."
2,3,gray,"(0.009765625, 0.010517900013934578)",3,"[13.645876, -3.3630786, -2.0767288, -0.8908882..."
3,4,gray,"(0.011048543456039806, 0.01188039556698871)",4,"[-21.007559, 0.9600556, 2.682488, -0.13888374,..."
4,5,gray,"(0.012352647110032733, 0.014084184669781208)",6,"[-3.995517, 1.8902166, -1.7816994, -0.9584407,..."


In [15]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [16]:
FINAL_DATA_NAME = 'approx1e5-agriVision-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [17]:
TRANSFORM = "wavelet"
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
                             
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [01:33<00:00, 48.35it/s]


,channel,layer,frequency,orientation,data
0,red,1,0.099611,L1,"[12.37854, 388.76587, -71.011154, -525.40607, ..."
1,red,2,0.110679,D,"[-56.85054, 10.112775, -65.61645, 3.6756237, -..."
2,red,2,0.110679,H,"[426.77856, 36.807053, 2.2234993, 41.115826, 3..."
3,red,2,0.110679,V,"[-458.58713, 52.57764, -45.266056, -4.2271876,..."
4,red,3,0.124514,D,"[-192.24008, 1.2320688, -6.463406, 114.53845, ..."


In [18]:
del channel_wv, group_data_map, group_data_map_size

In [19]:
TRANSFORM = "wavelet"
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


 31%|███▏      | 1416/4500 [00:32<01:11, 43.29it/s]


KeyboardInterrupt: 

In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
TRANSFORM = "wavelet"
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:04<00:00, 365.56it/s]


,channel,layer,frequency,orientation,data
0,blue,1,0.110679,L1,"[0.20727114, -33.177452, -23.717556, -104.1030..."
1,blue,2,0.124514,D,"[-26.155533, 6.1044655, 21.780111, 24.545807, ..."
2,blue,2,0.124514,H,"[-90.20362, -75.99399, 19.310474, -11.829856, ..."
3,blue,2,0.124514,V,"[76.4653, 12.720116, -29.556787, -10.373663, 0..."
4,blue,3,0.142301,D,"[53.602417, 3.668749, -16.509668, -0.57486, -1..."


In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
TRANSFORM = "wavelet"
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:04<00:00, 339.12it/s]


,channel,layer,frequency,orientation,data
0,gray,1,0.110679,L1,"[-6.1360106, 35.824085, -12.069928, -92.59161,..."
1,gray,2,0.124514,D,"[-20.23139, 5.5501356, 18.585161, 14.010576, -..."
2,gray,2,0.124514,H,"[-54.903618, -72.86237, 8.412701, -6.4397974, ..."
3,gray,2,0.124514,V,"[55.306328, 9.492829, -24.910969, -6.03024, 0...."
4,gray,3,0.142301,D,"[36.00795, 3.1733897, -9.826171, -1.8638246, -..."


In [ ]:
del channel_wv, group_data_map, group_data_map_size